In [147]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt


# Q1. Downloading the data


In [66]:
df_2023_01 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [77]:
df_2023_02 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet') 

Read the data for January. How many columns are there?

In [78]:
len(df_2023_01.columns)

19

# Q2. Computing duration


Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [79]:
df_2023_01['duration_seconds'] = (df_2023_01.tpep_dropoff_datetime - df_2023_01.tpep_pickup_datetime).dt.total_seconds() / 60

In [80]:
df_2023_02['duration_seconds'] = (df_2023_01.tpep_dropoff_datetime - df_2023_01.tpep_pickup_datetime).dt.total_seconds() / 60

In [81]:
df_2023_01.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration_seconds
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [82]:
df_2023_01.duration_seconds.std()

42.594351241920904

# Q3. Dropping outliers


Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [83]:
df_2023_01_filtered = df_2023_01[(df_2023_01['duration_seconds'] >= 60) & (df_2023_01['duration_seconds'] <= 60*60)]

In [116]:
df_2023_02_filtered = df_2023_02[(df_2023_02['duration_seconds'] >= 60) & (df_2023_02['duration_seconds'] <= 60*60)]

In [84]:
df_2023_01_filtered

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration_seconds
59,2,2023-01-01 00:22:39,2023-01-01 01:30:45,1.0,16.02,1.0,N,132,265,2,61.10,1.0,0.5,0.00,0.00,1.0,64.85,0.0,1.25,68.100000
383,2,2023-01-01 00:36:07,2023-01-02 00:17:13,1.0,1.20,1.0,N,234,234,1,11.40,1.0,0.5,0.00,0.00,1.0,16.40,2.5,0.00,1421.100000
533,2,2023-01-01 00:21:42,2023-01-01 23:18:28,2.0,3.05,1.0,N,48,140,1,19.80,1.0,0.5,3.72,0.00,1.0,28.52,2.5,0.00,1376.766667
546,2,2023-01-01 00:11:01,2023-01-01 23:37:57,1.0,1.68,1.0,N,138,129,1,10.70,6.0,0.5,3.89,0.00,1.0,23.34,0.0,1.25,1406.933333
760,1,2023-01-01 00:38:25,2023-01-01 01:38:39,1.0,13.70,1.0,N,161,200,1,70.90,3.5,0.5,15.80,3.00,1.0,94.70,2.5,0.00,60.233333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066030,1,2023-01-31 16:49:45,2023-01-31 17:56:16,NaN,0.00,NaN,None,163,132,0,64.00,2.5,0.5,19.03,6.55,1.0,98.08,NaN,NaN,66.516667
3066068,2,2023-01-31 17:38:06,2023-01-31 18:43:18,NaN,21.90,NaN,None,143,132,0,77.00,0.0,0.5,10.00,6.55,1.0,97.55,NaN,NaN,65.200000
3066081,2,2023-01-31 17:44:41,2023-01-31 18:48:49,NaN,18.57,NaN,None,237,132,0,77.00,0.0,0.5,16.20,0.00,1.0,97.20,NaN,NaN,64.133333
3066159,2,2023-01-31 17:30:32,2023-01-31 18:37:19,NaN,14.84,NaN,None,141,132,0,77.00,0.0,0.5,8.10,0.00,1.0,89.10,NaN,NaN,66.783333


In [85]:
len(df_2023_01_filtered) / len(df_2023_01)

0.007923004233123754

# Q4. One-hot encoding


Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [88]:
df_2023_01_filtered

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration_seconds
59,2,2023-01-01 00:22:39,2023-01-01 01:30:45,1.0,16.02,1.0,N,132,265,2,61.10,1.0,0.5,0.00,0.00,1.0,64.85,0.0,1.25,68.100000
383,2,2023-01-01 00:36:07,2023-01-02 00:17:13,1.0,1.20,1.0,N,234,234,1,11.40,1.0,0.5,0.00,0.00,1.0,16.40,2.5,0.00,1421.100000
533,2,2023-01-01 00:21:42,2023-01-01 23:18:28,2.0,3.05,1.0,N,48,140,1,19.80,1.0,0.5,3.72,0.00,1.0,28.52,2.5,0.00,1376.766667
546,2,2023-01-01 00:11:01,2023-01-01 23:37:57,1.0,1.68,1.0,N,138,129,1,10.70,6.0,0.5,3.89,0.00,1.0,23.34,0.0,1.25,1406.933333
760,1,2023-01-01 00:38:25,2023-01-01 01:38:39,1.0,13.70,1.0,N,161,200,1,70.90,3.5,0.5,15.80,3.00,1.0,94.70,2.5,0.00,60.233333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066030,1,2023-01-31 16:49:45,2023-01-31 17:56:16,NaN,0.00,NaN,None,163,132,0,64.00,2.5,0.5,19.03,6.55,1.0,98.08,NaN,NaN,66.516667
3066068,2,2023-01-31 17:38:06,2023-01-31 18:43:18,NaN,21.90,NaN,None,143,132,0,77.00,0.0,0.5,10.00,6.55,1.0,97.55,NaN,NaN,65.200000
3066081,2,2023-01-31 17:44:41,2023-01-31 18:48:49,NaN,18.57,NaN,None,237,132,0,77.00,0.0,0.5,16.20,0.00,1.0,97.20,NaN,NaN,64.133333
3066159,2,2023-01-31 17:30:32,2023-01-31 18:37:19,NaN,14.84,NaN,None,141,132,0,77.00,0.0,0.5,8.10,0.00,1.0,89.10,NaN,NaN,66.783333


In [117]:
categorical = ['PULocationID', 'DOLocationID']
df_2023_01_filtered[categorical] = df_2023_01_filtered[categorical].astype(str)
df_2023_02_filtered[categorical] = df_2023_02_filtered[categorical].astype(str)


/tmp/ipykernel_5979/3439382815.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_01_filtered[categorical] = df_2023_01_filtered[categorical].astype(str)
/tmp/ipykernel_5979/3439382815.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_02_filtered[categorical] = df_2023_02_filtered[categorical].astype(str)


In [118]:
df_2023_01_filtered['PU_DO'] = df_2023_01_filtered['PULocationID'] + '_' + df_2023_01_filtered['DOLocationID']
df_2023_02_filtered['PU_DO'] = df_2023_02_filtered['PULocationID'] + '_' + df_2023_02_filtered['DOLocationID']

/tmp/ipykernel_5979/1790551906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_01_filtered['PU_DO'] = df_2023_01_filtered['PULocationID'] + '_' + df_2023_01_filtered['DOLocationID']
/tmp/ipykernel_5979/1790551906.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_02_filtered['PU_DO'] = df_2023_02_filtered['PULocationID'] + '_' + df_2023_02_filtered['DOLocationID']


In [164]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_2023_01_filtered[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_2023_02_filtered[categorical].to_dict(orient='records')
X_val  = dv.transform(val_dicts)


In [165]:
X_train.shape

(24298, 4693)

In [166]:
X_val.shape

(23006, 4693)

# Q5. Training a model


Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [176]:
target = 'duration_seconds'
y_train = df_2023_01_filtered[target].values
y_true = df_2023_02_filtered[target].values

In [188]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [189]:
sqrt(mean_squared_error(y_train, y_pred))

194.39983503218173

# Q6. Evaluating the model


Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [190]:
target = 'duration_seconds'
y_train = df_2023_01_filtered[target].values
y_true = df_2023_02_filtered[target].values

In [191]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

In [192]:
sqrt(mean_squared_error(y_val, y_pred))

889.3905285119821